# 1. Load necessary modules

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import time

from skimage.feature import hog
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

from scipy.ndimage.measurements import label
from sklearn.model_selection import train_test_split


# 2. HOG - Histogram of Oriented Gradient
This function returns a vector of gradients which represent the image. It takes an image, orientation, pixels per cell, and cells per block as inputs, and returns the gradient for each cell.

In [ ]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# 3. Spatial Binning
This function resizes an image and converts the image to a 1D array feature vector in preparation for generating histograms of color.

In [ ]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# 4. Color Histogram
This function returns a histogram of color distributions within the image. It takes an image, number of bins, and bin range as an input and returns a histogram.

In [ ]:
# Define a function to compute color histogram features 
def color_hist(img, nbins=32, bins_range=(0, 1)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# 5. Extract Features
This function is part of the training pipeline. It extracts features from the training data by using the HOG and color histogram functions above to generate a feature vector based on the parameters passed as inputs to the function. The feature vector is used to train the classifier in the next function.

In [ ]:
# Define a function to extract features from a list of images
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

# 6. Sliding Window
This function generates "sliding windows" which split the image into small subsections which can be used to identify features (e.g. cars). The inputs include the image, a bound on the X and Y positions to search for features, window size, and window overlap. The function returns a list of windows which will be used to identify features in a later function.

In [ ]:
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    #nx_pix_per_step2 = np.int(xy_window[0]*0.5*(1 - xy_overlap[0]))
    #ny_pix_per_step2 = np.int(xy_window[1]*0.5*(1 - xy_overlap[1]))
    nx_pix_per_step3 = np.int(xy_window[0]*1.5*(1 - xy_overlap[0]))
    ny_pix_per_step3 = np.int(xy_window[1]*1.5*(1 - xy_overlap[1]))
    nx_pix_per_step4 = np.int(xy_window[0]*2*(1 - xy_overlap[0]))
    ny_pix_per_step4 = np.int(xy_window[1]*2*(1 - xy_overlap[1]))
    nx_pix_per_step5 = np.int(xy_window[0]*2.5*(1 - xy_overlap[0]))
    ny_pix_per_step5 = np.int(xy_window[1]*2.5*(1 - xy_overlap[1]))  
    
    # Compute the number of windows in x/y
    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1
    #nx_windows2 = np.int(xspan/nx_pix_per_step2) - 1
    #ny_windows2 = np.int(yspan/ny_pix_per_step2) - 1
    nx_windows3 = np.int(xspan/nx_pix_per_step3) - 1
    ny_windows3 = np.int(yspan/ny_pix_per_step3) - 1
    nx_windows4 = np.int(xspan/nx_pix_per_step4) - 1
    ny_windows4 = np.int(yspan/ny_pix_per_step4) - 1
    nx_windows5 = np.int(xspan/nx_pix_per_step5) - 1
    ny_windows5 = np.int(yspan/ny_pix_per_step5) - 1
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    #for ys in range(ny_windows2):
        #for xs in range(nx_windows2):
            # Calculate window position
            #startx = xs*nx_pix_per_step2 + x_start_stop[0]
            #endx = np.int(startx + xy_window[0]*0.5)
            #starty = ys*ny_pix_per_step2 + y_start_stop[0]
            #endy = np.int(starty + xy_window[1]*0.5)
            
            # Append window position to list
            #window_list.append(((startx, starty), (endx, endy)))
    for ys in range(ny_windows3):
        for xs in range(nx_windows3):
            # Calculate window position
            startx = xs*nx_pix_per_step3 + x_start_stop[0]
            endx = np.int(startx + xy_window[0]*1.5)
            starty = ys*ny_pix_per_step3 + y_start_stop[0]
            endy = np.int(starty + xy_window[1]*1.5)
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    for ys in range(ny_windows4):
        for xs in range(nx_windows4):
            # Calculate window position
            startx = xs*nx_pix_per_step4 + x_start_stop[0]
            endx = np.int(startx + xy_window[0]*2)
            starty = ys*ny_pix_per_step4 + y_start_stop[0]
            endy = np.int(starty + xy_window[1]*2)
    
    for ys in range(ny_windows5):
        for xs in range(nx_windows5):
            # Calculate window position
            startx = xs*nx_pix_per_step5 + x_start_stop[0]
            endx = np.int(startx + xy_window[0]*2.5)
            starty = ys*ny_pix_per_step5 + y_start_stop[0]
            endy = np.int(starty + xy_window[1]*2.5)            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# 7. Draw Boxes
This function draws boxes on the image to illustrate the features that have been identified as a match by the classifier.

In [ ]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

# 8. Single Image Features
This function performs the same function as "extract_features" in the training pipeline, but for a single image. It extracts features from the training data by using the HOG and color histogram functions above to generate a feature vector based on the parameters passed as inputs to the function. The feature vector is used to train the classifier in the next function.

In [ ]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# 9. Search Windows
This function returns the windows from the sliding windows function which the classifier identifies as a positive prediction. This is the core function used to identify vehicles within an image.

In [ ]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

# 10. Add Heat
This function generates a heatmap for an image by adding (1) to all pixels within a box identified by the "search_windows" function above. The heat map will be used to help filter out false positives.

In [ ]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes

# 11. Apply Threshold
This function applies a threshold to the heat map in order to help filter out false positives. And pixel with a value below the threshold will not be considered part of a positive detection.

In [ ]:
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


# 12. Draw Labeled Boxes
This function draws a box around the filtered hotspots on the original image, the last step in identifying the areas of a positive detection.

In [ ]:
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

# 13. Video Pipeline
This function takes an image as an input and returns the same image with filtered positive detections highlighted by a blue box.

In [ ]:
global hot_windows_prev
global hot_windows_prev_2
global hot_windows_prev_3
global hot_windows_prev_4

hot_windows_prev = [((0,0), (1,1))]
hot_windows_prev_2 = np.copy(hot_windows_prev)
hot_windows_prev_3 = np.copy(hot_windows_prev)
hot_windows_prev_4 = np.copy(hot_windows_prev)

def process_image(image):
    draw_image = np.copy(image)
    img_copy = np.copy(image)
    global hot_windows_prev
    global hot_windows_prev_2
    global hot_windows_prev_3
    global hot_windows_prev_4

    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    image = image.astype(np.float32)/255

    windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(96, 96), xy_overlap=(0.85, 0.85))

    hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    
    
    # Add heat to each box in box list, sum results with the previous four images to 
    # filter out false positives and smooth detection windows
    heat = add_heat(heat,hot_windows)
    heat = add_heat(heat, hot_windows_prev)
    heat = add_heat(heat, hot_windows_prev_2)
    heat = add_heat(heat, hot_windows_prev_3)
    heat = add_heat(heat, hot_windows_prev_4)
    
    hot_windows_prev_4 = np.copy(hot_windows_prev_3)
    hot_windows_prev_3 = np.copy(hot_windows_prev_2)
    hot_windows_prev_2 = np.copy(hot_windows_prev)
    hot_windows_prev = np.copy(hot_windows)
    
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,6)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(img_copy, labels)
    
    return draw_img

# 14. Training Code
This block of code is used to train the classifier and print an example image.

In [ ]:
# Read in cars and notcars
cars = []
notcars = []
images = glob.glob('non-vehicles/non-vehicles/*/*')
#images = glob.glob('non-vehicles_smallset/non-vehicles_smallset/*/*')
for image in images:
    notcars.append(image)
images = glob.glob('vehicles/vehicles/*/*')
#images = glob.glob('vehicles_smallset/vehicles_smallset/*/*')
for image in images:
    cars.append(image)

color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [300, None] # Min and max in y to search in slide_window()

car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.1, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
svc = LinearSVC()  
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

image = mpimg.imread('test_images/test1.jpg')
#image = mpimg.imread('bbox-example-image.jpg')
draw_image = np.copy(image)

# Uncomment the following line if you extracted training
# data from .png images (scaled 0 to 1 by mpimg) and the
# image you are searching is a .jpg (scaled 0 to 255)
image = image.astype(np.float32)/255

windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(96, 96), xy_overlap=(0.85, 0.85))

hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

plt.imshow(window_img)
plt.show()
cv2.imwrite('window_img.jpg', window_img)

# 15. Video
This block of code is used to run an image through the image pipeline and add the resulting image to a results video.

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'project_video_labeled.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)


HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))